# The Greeks

The greeks are the partial-derivatives of the Black-Scholes equation with respect to each variable. Greeks are closed-form formulas that represent the sensitivity of the option to the different underlying parameters.  So we are going to start with some theory and then we will code the greeks.

### Black–Scholes-Metron

to determine the price of vanilla Europrean options:

1. European options can only be exercised at expiration

2. No dividends are paid during the option's life

3. Market movements cannot be predicted

4. The risk-free rate and volatility are constant

5. Follows a lognormal distribution



For pricing a Call-Option:

$$\textbf{C} = S * N (d_1) - K * e^{-r_k,f *T}*N(d_2)$$

For pricing a Put-Option:

 $$\textbf{P} = K * e^{−r_k,f*T)}N(−d_2)−SN(−d_1)$$



##### where

$$
d_1 = \frac{\ln(\frac{S}{K}) + (r + \frac{stdev^2}{2})t}{s \cdot \sqrt{t}}
$$

$$
d_2 = d_1 - s \cdot \sqrt{t} = \frac{\ln(\frac{S}{K}) + (r - \frac{stdev^2}{2})t}{s \cdot \sqrt{t}}
$$

##### with 

$S$ the spot price of the asset at time t


$T$ the maturity of the option. Time to maturity is defined as T−t


$K$ strike price of the option


$r$ the risk-free interest rate, assumed to be constant between t and T


$\sigma$, volatility of underlying asset, the standard deviation of the asset returns

## Greeks

The greeks are the partial-derivatives of the Black-Scholes equation with respect to each variable. We will create credit spreads so we need the Theta $\Theta$, Delta $\Delta$ and the Gamma $\Gamma$. 

Credit Spread Ratios

$$Risk Ratio = \frac{Net \Theta} {\sqrt{Net \Delta^2 + Net \Gamma^2}}$$

$\Theta$ = the partial-derivative with respect to time until expiration and losing value per day


$\Delta$ = the first partial-derivative with respect to the underlying asset and shows the sensitivity of the option price to a move of the underlying spot price


$\Gamma$ = the 2nd partial-derivative with respect to the underlying asset


$$Cost Ratio = \frac{Net Credit}{Share Cost + Margin Required}$$



In [ ]:
import datetime
import re
import pandas as pd
import QuantLib as ql
import pandas as pd
import numpy as np
import yfinance as yf
from yahoo_fin.stock_info import get_quote_table

In [151]:
def option_data(ticker):
    
    opt_sym = yf.Ticker(ticker)
    info = get_quote_table(ticker)
    current_price = info["Quote Price"]
    yield_re = re.compile(r"\((?P<value>(\d+\.\d+))%\)")
    try:
        dividend_yield = float(
            yield_re.search(info["Forward Dividend & Yield"])["value"]
        )
    except (KeyError, ValueError, TypeError):
        dividend_yield = 0.0
    exps = opt_sym.options
    options_ = pd.DataFrame()
    def create_option(row):
        volatility = ql.BlackConstantVol(
            today,
            ql.UnitedStates(),
            row["volatility"],
            ql.Business252()
        )
        option = ql.VanillaOption(
            ql.PlainVanillaPayoff(ql.Option.Call, row["strike"]),
            exercise
        )
        process = ql.BlackScholesMertonProcess(
            ql.QuoteHandle(underlying),
            ql.YieldTermStructureHandle(dividendYield),
            ql.YieldTermStructureHandle(riskFreeRate),
            ql.BlackVolTermStructureHandle(volatility),
        )
        # Calculate it out from the last price
        imp_vol = option.impliedVolatility(row["lastPrice"], process)
        implied_volatility = ql.BlackConstantVol(
            today,
            ql.UnitedStates(),
            imp_vol,
            ql.Business252()
        )
        process = ql.BlackScholesMertonProcess(
            ql.QuoteHandle(underlying),
            ql.YieldTermStructureHandle(dividendYield),
            ql.YieldTermStructureHandle(riskFreeRate),
            ql.BlackVolTermStructureHandle(implied_volatility),
        )
        option.setPricingEngine(
            ql.FdBlackScholesVanillaEngine(process, 1000, 1000)
        )
        return {
            "Name": row["contractSymbol"],
            "Strike": row["strike"],
            "Last": row["lastPrice"],
            "Bid": row["bid"],
            "Ask": row["ask"],
            "NPV": option.NPV(),
            "Delta": option.delta(),
            "Gamma": option.gamma(),
            "Theta": option.theta() / 365,
            "Volatility": imp_vol * 100
        }
        options = calls.apply(create_option, axis=1, result_type="expand")
        
    for e in exps:
        expiration = pd.to_datetime(e) + datetime.timedelta(days = 1)
        opt = opt_sym.option_chain(e)   
        calls = pd.DataFrame().append(opt.calls).append(opt.puts)
        # Setup instruments for Black-Scholes pricing
        today = ql.Date.todaysDate()
        underlying = ql.SimpleQuote(current_price)
        exercise = ql.AmericanExercise(
            today,
        ql.Date(expiration.day, expiration.month, expiration.year))
        dividendYield = ql.FlatForward(
            today, dividend_yield, ql.Actual360()
        )
        riskFreeRate = ql.FlatForward(today, 0.0008913, ql.Actual360())
        # Filter down to only OTM strikes
        calls = calls[calls["strike"] >= current_price * 1.025]
        calls = calls[calls["strike"] <= current_price * 1.10]
        # Parse out implied volatility
        calls = calls.assign(
        volatility=calls["impliedVolatility"],
        )
        options = calls.apply(create_option, axis=1, result_type="expand")
        options_ = options_.append(options, ignore_index=True) 
    return options_

In [152]:
ticker = "AAPL"
options=option_data(ticker)

In [158]:
options.head()

,Name,Strike,Last,Bid,Ask,NPV,Delta,Gamma,Theta,Volatility
0,AAPL210319C00125000,125.0000,0.6600,0.6700,0.6800,0.6611,0.2063,0.0499,-0.1048,38.3018
1,AAPL210319C00126000,126.0000,0.4800,0.4800,0.5000,0.4810,0.1604,0.0425,-0.0903,38.0977
2,AAPL210319C00127000,127.0000,0.3500,0.3400,0.3600,0.3508,0.1236,0.0353,-0.0765,38.1509
3,AAPL210319C00127500,127.5000,0.3000,0.2900,0.3100,0.3007,0.1084,0.0320,-0.0701,38.2766
4,AAPL210319C00128000,128.0000,0.2500,0.2500,0.2800,0.2506,0.0933,0.0287,-0.0627,38.1503


### Minimizing Risk and Maximizing Profits

In [154]:
# This code is a continuation of the same file from above
# Pair up options with the next available option
pairs = pd.concat(
    [
        options.add_suffix("_1"),
        options.shift(-1).add_suffix("_2")
    ],
    axis=1
)
pairs = pairs[pairs["Name_2"].notna()]

In [155]:
def maximize_theta(row):
    bid, ask = row["Bid_1"], row["Ask_2"]
    strike_1, strike_2 = row["Strike_1"], row["Strike_2"]
    delta_1, delta_2 = row["Delta_1"], row["Delta_2"]
    gamma_1, gamma_2 = row["Gamma_1"], row["Gamma_2"]
    theta_1, theta_2 = row["Theta_1"], row["Theta_2"]    
    def calculate_values(sell):        
        buy = round(gamma_1 * sell / gamma_2)
        credit = (bid * sell * 100) - (ask * buy * 100) - ((sell + buy) * 0.65)
        shares = -1 * round(delta_2 * buy * 100 - delta_1 * sell * 100)
        delta = delta_2 * buy * 100 - delta_1 * sell * 100 + shares
        gamma = gamma_2 * buy * 100 - gamma_1 * sell * 100
        theta = theta_2 * buy * 100 - theta_1 * sell * 100
        share_cost = shares * current_price
        margin = strike_2 * buy * 100 - strike_1 * sell * 100
        return {
            "Sell Contract": f"{row['Strike_1']} @ {bid}",
            "Sell Amount": sell,
            "Buy Contract": f"{row['Strike_2']} @ {ask}",
            "Buy Amount": buy,
            "Shares": shares,
            "Share Cost": share_cost,
            "Margin": margin,
            "Credit": credit,
            "Cost Ratio": credit / (share_cost + margin),
            "Net Delta": delta,
            "Net Gamma": gamma,
            "Net Theta": theta,
            "Risk Ratio": theta / (delta ** 2 + gamma ** 2) ** 0.5,
        }    
    trades = []
    sell = 1
    values = calculate_values(sell)
    while (abs(values["Share Cost"]) + abs(values["Margin"])) < 2500:
        if values["Shares"] > 0 and values["Credit"] > 0:
            trades.append(values)
        sell += 1
        values = calculate_values(sell)
    
    if trades:
        results = pd.DataFrame.from_records(trades)
        results.sort_values(
            by=["Risk Ratio", "Cost Ratio"],
            ascending=False,
            inplace=True
        )
        return results.iloc[0]
    return None

In [156]:
results = pairs.apply(maximize_theta, axis=1, result_type="expand")
results = results[results["Shares"].notna()]
print(results)

     Sell Contract  Sell Amount   Buy Contract  Buy Amount  Shares  \
0     125.0 @ 0.67       2.0000    126.0 @ 0.5      2.0000  9.0000   
1     126.0 @ 0.48       2.0000   127.0 @ 0.36      2.0000  7.0000   
2     127.0 @ 0.34       2.0000   127.5 @ 0.31      2.0000  3.0000   
4     128.0 @ 0.25       1.0000   129.0 @ 0.21      1.0000  2.0000   
5     129.0 @ 0.19       2.0000   130.0 @ 0.16      2.0000  3.0000   
..             ...          ...            ...         ...     ...   
142   125.0 @ 18.7       1.0000   130.0 @ 17.2      1.0000  3.0000   
145  130.0 @ 25.55       2.0000  125.0 @ 21.25      2.0000  3.0000   
146   125.0 @ 20.9       1.0000  130.0 @ 19.45      1.0000  3.0000   
149  130.0 @ 27.35       2.0000   125.0 @ 22.4      2.0000  3.0000   
150   125.0 @ 22.2       1.0000  130.0 @ 20.65      1.0000  3.0000   

     Share Cost      Margin   Credit  Cost Ratio  Net Delta  Net Gamma  \
0    1,089.2700    200.0000  31.4000      0.0244    -0.1840    -1.4879   
1      847.

In [157]:
results

,Sell Contract,Sell Amount,Buy Contract,Buy Amount,Shares,Share Cost,Margin,Credit,Cost Ratio,Net Delta,Net Gamma,Net Theta,Risk Ratio
0,125.0 @ 0.67,2.0000,126.0 @ 0.5,2.0000,9.0000,"1,089.2700",200.0000,31.4000,0.0244,-0.1840,-1.4879,2.9064,1.9386
1,126.0 @ 0.48,2.0000,127.0 @ 0.36,2.0000,7.0000,847.2100,200.0000,21.4000,0.0204,-0.3621,-1.4377,2.7629,1.8636
2,127.0 @ 0.34,2.0000,127.5 @ 0.31,2.0000,3.0000,363.0900,100.0000,3.4000,0.0073,-0.0347,-0.6669,1.2660,1.8956
4,128.0 @ 0.25,1.0000,129.0 @ 0.21,1.0000,2.0000,242.0600,100.0000,2.7000,0.0079,0.2003,-0.5066,0.7358,1.3506
5,129.0 @ 0.19,2.0000,130.0 @ 0.16,2.0000,3.0000,363.0900,200.0000,3.4000,0.0060,-0.4288,-0.9192,1.8872,1.8606
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,125.0 @ 18.7,1.0000,130.0 @ 17.2,1.0000,3.0000,363.0900,500.0000,148.7000,0.1723,-0.1834,-0.0264,-0.0030,-0.0162
145,130.0 @ 25.55,2.0000,125.0 @ 21.25,2.0000,3.0000,363.0900,"-1,000.0000",857.4000,-1.3462,-0.2029,0.2733,0.5950,1.7480
146,125.0 @ 20.9,1.0000,130.0 @ 19.45,1.0000,3.0000,363.0900,500.0000,143.7000,0.1665,0.0518,-0.0165,-0.0009,-0.0174
149,130.0 @ 27.35,2.0000,125.0 @ 22.4,2.0000,3.0000,363.0900,"-1,000.0000",987.4000,-1.5503,-0.1164,0.2540,0.2887,1.0330
